Time series to Supervised
==========

1 Using shift
---

In [4]:
import pandas as pd

df = pd.DataFrame()
df['t'] = [x for x in range(10)]
df

,t
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [3]:
df['t-1'] = df['t'].shift(1)
df

,t,t-1
0,0,NaN
1,1,0.0
2,2,1.0
3,3,2.0
4,4,3.0
5,5,4.0
6,6,5.0
7,7,6.0
8,8,7.0
9,9,8.0


In [5]:
# Technically, in time series forecasting terminology the current time (t) 
# and future times (t+1, t+n) are forecast times 
# and past observations (t-1, t-n) are used to make forecasts.

df['t'] = [x for x in range(10)]
df['t+1'] = df['t'].shift(-1)
df

,t,t+1
0,0,1.0
1,1,2.0
2,2,3.0
3,3,4.0
4,4,5.0
5,5,6.0
6,6,7.0
7,7,8.0
8,8,9.0
9,9,NaN


2 The series_to_supervised() Function
--------

In [17]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg

In [19]:
# var(t-1)의 값이 var1(t)값을 예상하기 위한 input값이 된다!!
values = [x for x in range(10)]
data = series_to_supervised(values, dropnan=False)
data

,var1(t-1),var1(t)
0,NaN,0
1,0.0,1
2,1.0,2
3,2.0,3
4,3.0,4
5,4.0,5
6,5.0,6
7,6.0,7
8,7.0,8
9,8.0,9


In [21]:
data2 = series_to_supervised(values, 3, dropnan=False)
data2

,var1(t-3),var1(t-2),var1(t-1),var1(t)
0,NaN,NaN,NaN,0
1,NaN,NaN,0.0,1
2,NaN,0.0,1.0,2
3,0.0,1.0,2.0,3
4,1.0,2.0,3.0,4
5,2.0,3.0,4.0,5
6,3.0,4.0,5.0,6
7,4.0,5.0,6.0,7
8,5.0,6.0,7.0,8
9,6.0,7.0,8.0,9


3 Multi-Step or Sequence Forecasting
---------

In [24]:
data3 = series_to_supervised(values, 2,2, dropnan=False)
data3

,var1(t-2),var1(t-1),var1(t),var1(t+1)
0,NaN,NaN,0,1.0
1,NaN,0.0,1,2.0
2,0.0,1.0,2,3.0
3,1.0,2.0,3,4.0
4,2.0,3.0,4,5.0
5,3.0,4.0,5,6.0
6,4.0,5.0,6,7.0
7,5.0,6.0,7,8.0
8,6.0,7.0,8,9.0
9,7.0,8.0,9,NaN


4 Multivariate Forecasting
----

In [26]:
raw = pd.DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
raw.head()

,ob1,ob2
0,0,50
1,1,51
2,2,52
3,3,53
4,4,54


In [28]:
values = raw.values
data4 = series_to_supervised(values, dropnan=False)
data4

,var1(t-1),var2(t-1),var1(t),var2(t)
0,NaN,NaN,0,50
1,0.0,50.0,1,51
2,1.0,51.0,2,52
3,2.0,52.0,3,53
4,3.0,53.0,4,54
5,4.0,54.0,5,55
6,5.0,55.0,6,56
7,6.0,56.0,7,57
8,7.0,57.0,8,58
9,8.0,58.0,9,59


In [30]:
data5 = series_to_supervised(values, 1, 2, dropnan=False)
data5

,var1(t-1),var2(t-1),var1(t),var2(t),var1(t+1),var2(t+1)
0,NaN,NaN,0,50,1.0,51.0
1,0.0,50.0,1,51,2.0,52.0
2,1.0,51.0,2,52,3.0,53.0
3,2.0,52.0,3,53,4.0,54.0
4,3.0,53.0,4,54,5.0,55.0
5,4.0,54.0,5,55,6.0,56.0
6,5.0,55.0,6,56,7.0,57.0
7,6.0,56.0,7,57,8.0,58.0
8,7.0,57.0,8,58,9.0,59.0
9,8.0,58.0,9,59,NaN,NaN


### Reference : https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/